In [8]:
%pip install -U langgraph langchain_community langchain_anthropic langchain_experimental

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
%pip install python-dotenv langchain_groq langchain_ollama


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

import getpass
import os
from dotenv import find_dotenv, load_dotenv
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
groq_api_key = os.getenv("groq_api_key")
os.environ["groq_api_key"] = groq_api_key 

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("groq_api_key")

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
tavily_api_key = os.getenv("tavily_api_key")
os.environ["tavily_api_key"] = tavily_api_key

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("tavily_api_key")

In [3]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

tavily_tool = TavilySearchResults(max_results=5)

# This executes code locally, which can be unsafe
repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code and do math. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
    return result_str

<>:23: SyntaxWarning: invalid escape sequence '\`'
<>:23: SyntaxWarning: invalid escape sequence '\`'
<>:23: SyntaxWarning: invalid escape sequence '\`'
<>:23: SyntaxWarning: invalid escape sequence '\`'
C:\Users\paulo\AppData\Local\Temp\ipykernel_3712\2391770398.py:23: SyntaxWarning: invalid escape sequence '\`'
  result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"
C:\Users\paulo\AppData\Local\Temp\ipykernel_3712\2391770398.py:23: SyntaxWarning: invalid escape sequence '\`'
  result_str = f"Successfully executed:\n\`\`\`python\n{code}\n\`\`\`\nStdout: {result}"


In [4]:
from typing import Literal
from typing_extensions import TypedDict

from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, END
from langgraph.types import Command


members = ["researcher", "coder", "expenses_manager"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = members + ["FINISH"]

system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    f" following workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)


class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""

    next: Literal[*options]


#llm = ChatAnthropic(model="claude-3-5-sonnet-latest")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama-3.3-70b-Versatile")
llm_tool=ChatGroq(groq_api_key=groq_api_key,model_name="Llama-3.3-70b-Versatile")
#MODEL_NAME = "deepseek-r1:latest"

#MODEL_NAME = "gemma3:4b"
#MODEL_NAME = "llama3.2:latest"
#llm = ChatOllama(model=MODEL_NAME, temperature=0.5)

#MODEL_NAME = "llama3.2:latest"
#llm_tool = ChatOllama(model=MODEL_NAME, temperature=0.5)
class State(MessagesState):
    next: str

    
call_count = 0
call_count_max = 2
def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
    global call_count  # Now it refers to the global variable
    global call_count_max

    call_count = call_count +1

    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    if call_count  == call_count_max:
        goto = END

    return Command(goto=goto, update={"next": goto})

In [5]:
result = llm.invoke("state")

In [6]:
result

AIMessage(content='The term \'state\' can refer to different concepts depending on the context. Here are a few possible meanings:\n\n1. **Government or country**: A state is a sovereign entity with its own government, territory, and population. It has the power to govern its citizens and make decisions on their behalf.\n2. **Condition or status**: A state can also refer to a particular condition or status of something or someone. For example, "the state of the economy" or "the state of mind."\n3. **Region or province**: In some countries, a state is a subdivision of the country, similar to a province or region. For example, the United States is divided into 50 states.\n4. **State of matter**: In physics, a state refers to the physical condition of a substance, such as solid, liquid, or gas.\n5. **Mental or emotional state**: A state can also refer to a person\'s mental or emotional condition, such as a state of happiness, sadness, or anxiety.\n\nWithout more context, it\'s difficult to

In [7]:
result

AIMessage(content='The term \'state\' can refer to different concepts depending on the context. Here are a few possible meanings:\n\n1. **Government or country**: A state is a sovereign entity with its own government, territory, and population. It has the power to govern its citizens and make decisions on their behalf.\n2. **Condition or status**: A state can also refer to a particular condition or status of something or someone. For example, "the state of the economy" or "the state of mind."\n3. **Region or province**: In some countries, a state is a subdivision of the country, similar to a province or region. For example, the United States is divided into 50 states.\n4. **State of matter**: In physics, a state refers to the physical condition of a substance, such as solid, liquid, or gas.\n5. **Mental or emotional state**: A state can also refer to a person\'s mental or emotional condition, such as a state of happiness, sadness, or anxiety.\n\nWithout more context, it\'s difficult to

In [9]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent


research_agent = create_react_agent(
    llm_tool, tools=[tavily_tool], prompt="You are a researcher. DO NOT do any math."
)


def research_node(state: State) -> Command[Literal["supervisor"]]:
    #print("state:",state)
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )


# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION, WHICH CAN BE UNSAFE WHEN NOT SANDBOXED
code_agent = create_react_agent(llm, tools=[python_repl_tool])


def code_node(state: State) -> Command[Literal["supervisor"]]:
    #print("state:",state)
    result = code_agent.invoke(state)

    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor",
    )


import json
# Load the JSON file
file_path = "monthly_expenses.json"  # Replace with the correct path to your file
with open(file_path, "r") as file:
    data = json.load(file)

# Convert the JSON data to a string
monthly_expenses_json_string = json.dumps(data, indent=4)  # Pretty-printed string

expenses_manager_agent = create_react_agent(llm, tools=[],prompt=f"You are a expenses manager. you data is solely based here{monthly_expenses_json_string}")

def expenses_manager_node(state: State) -> Command[Literal["supervisor"]]:

    result = expenses_manager_agent.invoke(state)

    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="expenses_manager")
            ]
         
        },
        goto="supervisor",
    )


builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", code_node)
builder.add_node("expenses_manager", expenses_manager_node)
graph = builder.compile()

In [15]:
from IPython.display import display, Image

#display(Image(graph.get_graph().draw_mermaid_png()))

In [16]:
call_count = 0
call_count_max = 5
events = graph.stream(
    {"messages": [("user", "What's the square root of 42?")]}, subgraphs=True
)

In [17]:
my_array = []
for s in events:
    print(s)
    my_array.append(s)
    print("----")

((), {'supervisor': {'next': 'researcher'}})
----
state: {'messages': [HumanMessage(content="What's the square root of 42?", additional_kwargs={}, response_metadata={}, id='5c9577ff-7b4f-4313-ad00-802679db68a4')], 'next': 'researcher'}
(('researcher:2720ff28-132b-8c39-0a78-e5e8e3e005a6',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:latest', 'created_at': '2025-04-02T02:26:18.887474Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4526373000, 'load_duration': 3903171500, 'prompt_eval_count': 205, 'prompt_eval_duration': 229000000, 'eval_count': 25, 'eval_duration': 387000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-1e028666-480a-4cd2-96fe-e087cced3ffe-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'square root of 42'}, 'id': 'ace1d6f6-fbf1-4e13-9124-ab8ee6dab9dc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 205, 'output_tokens': 25

In [21]:
agents = []
j = -2
for i in range(len(my_array)):
    
    try:
        if my_array[i][1]['supervisor']['next']:  # Check if 'next' key exists and is not empty
            

            if (j == -2):
                agent =  my_array[i][1]['supervisor']['next']
                print("AGENT:", agent)


            j = j+1
           
            agent =  my_array[i][1]['supervisor']['next']
         
            agents.append(agent)

            #agent = agent.append(my_array[i][1]['supervisor']['next'])
            print(my_array[i - 1][1][agents[j]]["messages"][0].content)
            print("-----------")

            print("AGENT:", agent)
            
            
            
    except (KeyError, IndexError, TypeError):
        pass


AGENT: researcher
The square root of 42 is approximately 6.481, which cannot be reduced further and is an irrational number.
-----------
AGENT: researcher
I apologize for the mistake. It seems like I don't have direct access to the math library. However, I can try searching for the square root of 42.

According to my search results, the square root of 42 is approximately 6.481, which cannot be reduced further and is an irrational number.
-----------
AGENT: researcher
The square root of 42 is approximately 6.481, which cannot be reduced further and is an irrational number.
-----------
AGENT: researcher
The square root of 42 is approximately 6.481, which cannot be reduced further and is an irrational number.
-----------
AGENT: __end__


In [24]:
call_count = 0
call_count_max = 5
events = graph.stream(
    {"messages": [("user", "What's the square root of 42?")]}, subgraphs=True
)

my_array = []
for s in events:
    
    my_array.append(s)

agents = []
j = -2
for i in range(len(my_array)):
    
    try:
        if my_array[i][1]['supervisor']['next']:  # Check if 'next' key exists and is not empty
            

            if (j == -2):
                agent =  my_array[i][1]['supervisor']['next']
                print("AGENT:", agent)


            j = j+1
           
            agent =  my_array[i][1]['supervisor']['next']
         
            agents.append(agent)

            #agent = agent.append(my_array[i][1]['supervisor']['next'])
            print(my_array[i - 1][1][agents[j]]["messages"][0].content)
            print("-----------")

            print("AGENT:", agent)
            
            
            
    except (KeyError, IndexError, TypeError):
        pass

AGENT: researcher
It seems like I need to use the `tavily` search results instead of the `math.sqrt` function. Let me try again.

Unfortunately, it looks like 42 is not a perfect square, and its square root cannot be expressed as a simple integer or fraction. If you'd like, I can provide an approximate decimal value for you!
-----------
AGENT: researcher
The square root of 42 is approximately 6.481.
-----------
AGENT: researcher
The square root of 42 is approximately 6.481.
-----------
AGENT: __end__


In [10]:
def multi_agent_bot(prompt):
    global call_count
    global call_count_max

    call_count = 0
    call_count_max = 5
    events = graph.stream(
        {"messages": [("user", prompt)]}, subgraphs=True
    )

    my_array = []
    for s in events:
        print(s)
        my_array.append(s)

    agents = []
    j = -2
    for i in range(len(my_array)):
        
        try:
            if my_array[i][1]['supervisor']['next']:  # Check if 'next' key exists and is not empty
                

                if (j == -2):
                    agent =  my_array[i][1]['supervisor']['next']
                    print("AGENT:", agent)


                j = j+1
            
                agent =  my_array[i][1]['supervisor']['next']
            
                agents.append(agent)

                #agent = agent.append(my_array[i][1]['supervisor']['next'])
                print(my_array[i - 1][1][agents[j]]["messages"][0].content)
                print("-----------")

                print("AGENT:", agent)
                
                
                
        except (KeyError, IndexError, TypeError):
            pass

In [12]:
prompt = "what is average age of dog search the internet then give me a summary of the expenses"
multi_agent_bot(prompt)

((), {'supervisor': {'next': 'researcher'}})
(('researcher:5c5d5c4d-c32a-8794-c3d3-d4abacf1d84b',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6359', 'function': {'arguments': '{"query": "average age of dog"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_e5p8', 'function': {'arguments': '{"query": "expenses of owning a dog"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 306, 'total_tokens': 348, 'completion_time': 0.152727273, 'prompt_time': 0.017187575, 'queue_time': 0.24229918299999997, 'total_time': 0.169914848}, 'model_name': 'Llama-3.3-70b-Versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-da8b30f7-7e07-468e-a821-b6c924aedff3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'average age of dog'}, 'id': 'call_6359', 'type': 'tool_

In [11]:
prompt = " give me a summary of the expenses"
multi_agent_bot(prompt)

((), {'supervisor': {'next': 'expenses_manager'}})
(('expenses_manager:bb4ef7db-8651-3aa0-f980-80faee2f009d',), {'agent': {'messages': [AIMessage(content='Based on the provided data, here is a summary of the expenses:\n\n**Total Expenses by Year:**\n\n* 2023: \n  - Food: 94,300\n  - Transpo: 27,300\n  - Daily: 17,400\n  - Total: 139,000\n* 2024: \n  - Food: 100,400\n  - Transpo: 29,750\n  - Daily: 19,550\n  - Total: 149,700\n\n**Total Expenses by Category (Both Years):**\n\n* Food: 194,700\n* Transpo: 57,050\n* Daily: 36,950\n* Total: 288,700\n\n**Average Monthly Expenses by Category (Both Years):**\n\n* Food: 8,122\n* Transpo: 2,377\n* Daily: 1,546\n* Total: 12,045\n\n**Highest Monthly Expenses by Category (Both Years):**\n\n* Food: 9,500 (December 2024)\n* Transpo: 3,200 (December 2024)\n* Daily: 2,200 (December 2024)\n* Total: 15,900 (December 2024)\n\n**Lowest Monthly Expenses by Category (Both Years):**\n\n* Food: 7,500 (February 2023)\n* Transpo: 2,300 (February 2023)\n* Daily: 1

In [12]:
data 

{'2023': {'January': {'Food': 8000, 'Transpo': 2500, 'Daily': 1500},
  'February': {'Food': 7500, 'Transpo': 2300, 'Daily': 1400},
  'March': {'Food': 8500, 'Transpo': 2600, 'Daily': 1600},
  'April': {'Food': 8000, 'Transpo': 2400, 'Daily': 1500},
  'May': {'Food': 7800, 'Transpo': 2500, 'Daily': 1450},
  'June': {'Food': 8200, 'Transpo': 2600, 'Daily': 1500},
  'July': {'Food': 8500, 'Transpo': 2700, 'Daily': 1550},
  'August': {'Food': 8700, 'Transpo': 2800, 'Daily': 1600},
  'September': {'Food': 8100, 'Transpo': 2400, 'Daily': 1500},
  'October': {'Food': 8300, 'Transpo': 2500, 'Daily': 1550},
  'November': {'Food': 8600, 'Transpo': 2600, 'Daily': 1600},
  'December': {'Food': 9000, 'Transpo': 3000, 'Daily': 2000}},
 '2024': {'January': {'Food': 8500, 'Transpo': 2600, 'Daily': 1700},
  'February': {'Food': 8000, 'Transpo': 2500, 'Daily': 1600},
  'March': {'Food': 8700, 'Transpo': 2700, 'Daily': 1750},
  'April': {'Food': 8500, 'Transpo': 2600, 'Daily': 1700},
  'May': {'Food': 84

In [13]:
def calculate_totals(data):
    yearly_totals = {}
    for year, months in data.items():
        food_total = sum(month["Food"] for month in months.values())
        transpo_total = sum(month["Transpo"] for month in months.values())
        daily_total = sum(month["Daily"] for month in months.values())
        
        yearly_totals[year] = {
            "Total Food": food_total,
            "Total Transpo": transpo_total,
            "Total Daily": daily_total
        }
    return yearly_totals

result = calculate_totals(data)
print(json.dumps(result, indent=4))


{
    "2023": {
        "Total Food": 99200,
        "Total Transpo": 30900,
        "Total Daily": 18750
    },
    "2024": {
        "Total Food": 104800,
        "Total Transpo": 32750,
        "Total Daily": 21100
    }
}


#### the value up is correct (also check total food 2023 by calculator manually)meaning the llm agent is false, not reliable

In [11]:
prompt = "give me a summary of the expenses"
multi_agent_bot(prompt)

((), {'supervisor': {'next': 'expenses_manager'}})
(('expenses_manager:bcafae3f-8688-2aa7-de1b-9745e1c81ebb',), {'agent': {'messages': [AIMessage(content='Based on the provided data, here is a summary of the expenses:\n\n**Total Expenses by Year:**\n\n* 2023: \n  - Food: 93,800\n  - Transpo: 28,200\n  - Daily: 17,400\n  - Total: 139,400\n* 2024: \n  - Food: 100,400\n  - Transpo: 30,350\n  - Daily: 19,800\n  - Total: 150,550\n\n**Total Expenses by Category:**\n\n* Food: 194,200\n* Transpo: 58,550\n* Daily: 37,200\n* Total: 290,000 (for both years)\n\n**Average Monthly Expenses:**\n\n* 2023: \n  - Food: 7,817\n  - Transpo: 2,350\n  - Daily: 1,450\n  - Total: 11,617\n* 2024: \n  - Food: 8,367\n  - Transpo: 2,529\n  - Daily: 1,650\n  - Total: 12,546\n\n**Highest and Lowest Expenses:**\n\n* Highest Food Expense: December 2024 (9,500)\n* Highest Transpo Expense: December 2024 (3,200)\n* Highest Daily Expense: December 2024 (2,200)\n* Lowest Food Expense: February 2023 (7,500)\n* Lowest Trans